In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import metrics
import pandas
import glob
import xml.etree.ElementTree as ET
import nltk
import math
import numpy
import warnings

warnings.filterwarnings("ignore")

---
# 1. Extraindo trechos anotados dos XMLs

### 1.1. Adiquirindo a raiz de cada XML

In [2]:
glob_path = 'xml_batch1/*.xml' # Caminho até os XMLs
roots = []
for xml in glob.glob(glob_path):
    tree = ET.parse(xml)
    roots.append(tree.getroot())

### 1.2. Iremos agora iterar sobre a raiz de cada arquivo buscando pelas relações contidas nelas. Em seguida iremos usar as relações para compor cada linha do CSV.
Antes é importante entender a estrutura XML da anotação de uma relação:
```xml
<relation id="R1">
    <infon key="type">Ato_Tornado_Sem_Efeito_Exo_Nom</infon>
    <infon key="annotator">tatiana_franco</infon>
    <infon key="updated_at">2020-12-22T02:09:50Z</infon>
    <node refid="4192" role=""/>
    <node refid="4193" role=""/>
    <node refid="4194" role=""/>
    <node refid="4195" role=""/>
    <node refid="4196" role=""/>
    <node refid="4197" role=""/>
    <node refid="4198" role=""/>
    <node refid="4199" role=""/>
    <node refid="118512" role=""/>
    <node refid="118513" role=""/>
    <node refid="118514" role=""/>
</relation>
```
> Cada nó(node) representa um anotação. Basta recuperar o *refid* de cada nó e procurar pelo ID de sua respectiva anotação.

> Utilize a tag *infon* com *key=type* para identificar o ato anotado

```xml
<annotation id="4192">
    <infon key="type">tipo_documento</infon>
    <infon key="identifier"></infon>
    <infon key="annotator">lindeberg</infon>
    <infon key="updated_at">2020-12-22T00:41:28Z</infon>
    <location offset="385761" length="7"/>
    <text>Decreto</text>
</annotation>
```
> Novamente utilize *infon* com *key=type* para identificar qual o tipo de anotação e a tag *text* para resgatar o texto da anotação


In [3]:
# Esse bloco deve demorar de alguns segundos a 1 minuto
atos_csv_dict = {}

for root in roots: # Intera sobre as raizes
    for relation in root.findall(".//relation"):                            # Intera sobre as relações.
        row_act = {}
        type_relation = relation.find('.//infon[@key="type"]').text
        for node in relation.findall('node'):                               # Intera sobre os nós(anotações) da relação.
            ref_id = node.get('refid')
            annotation = root.find(f'.//annotation[@id="{ref_id}"]')        # Encontra anotação.
            type_annotation = annotation.find('.//infon[@key="type"]').text # Encontra tipo da anotação.
            text_annotation = annotation.find('text').text                  # Encontra texto da anotação.
            row_act[type_annotation] = text_annotation
        
        if type_relation not in atos_csv_dict:                              # Checa se a tabela já existe, caso contrário, cria uma.
            atos_csv_dict[type_relation] = pandas.DataFrame()
            
        atos_csv_dict[type_relation] = atos_csv_dict[type_relation].append(row_act, ignore_index=True)

Como resultado final temos um dicionario de CSVs. Cada chave do dicionário é referente a um ato e aponta para seu respectivo dataframe.

---

# 2. Exploração e validação da integridade dos dados

### 2.2. Vamos checar agora quantos atos temos e se os Dataframes estão coerentes.

In [4]:
print(atos_csv_dict.keys())
print("Quantidade de atos: " + str(len(atos_csv_dict.keys())))

dict_keys(['Ato_Retificacao_Comissionado', 'Ato_Tornado_Sem_Efeito_Exo_Nom', 'Ato_Exoneracao_Comissionado', 'Ato_Nomeacao_Comissionado', 'Ato_Nomeacao_Efetivo', 'Ato_Abono_Permanencia', 'Ato_Substituicao', 'Ato_Tornado_Sem_Efeito_Apo', 'Ato_Retificacao_Efetivo', 'Ato_Cessao', 'Ato_Reversao', 'Ato_Exoneracao_Efetivo'])
Quantidade de atos: 12


### 2.3. Temos todos os 12 atos presentes no dicionário. Checaremos agora o tamanho e os campos de cada dataframe.

In [5]:
for key in atos_csv_dict.keys():
    print("Ato: " + key)
    print("Campos: ", end='')
    print(atos_csv_dict[key].columns)
    print("Tamanho: " + str(len(atos_csv_dict[key])))
    print('----------------')

Ato: Ato_Retificacao_Comissionado
Campos: Index(['Ato_Retificacao_Comissionado', 'data_dodf', 'informacao_corrigida',
       'informacao_errada', 'nome', 'numero_documento', 'numero_dodf',
       'pagina_dodf', 'tipo_documento', 'data_documento', 'tipo_ato',
       'lotacao', 'matricula', 'a_pedido_ou_nao'],
      dtype='object')
Tamanho: 50
----------------
Ato: Ato_Tornado_Sem_Efeito_Exo_Nom
Campos: Index(['Ato_Tornado_Sem_Efeito_Exo_Nom', 'cargo_comissionado',
       'data_documento', 'data_dodf', 'hierarquia_lotacao', 'nome',
       'numero_dodf', 'orgao', 'pagina_dodf', 'simbolo', 'tipo_documento',
       'matricula', 'cargo_efetivo', 'motivo', 'matricula_SIAPE', 'lotacao'],
      dtype='object')
Tamanho: 54
----------------
Ato: Ato_Exoneracao_Comissionado
Campos: Index(['Ato_Exoneracao_Comissionado', 'cargo_comissionado',
       'hierarquia_lotacao', 'motivo', 'nome', 'orgao', 'simbolo',
       'a_pedido_ou_nao', 'vigencia', 'cargo_efetivo', 'matricula',
       'matricula_SIAPE'

### 2.4. Extraido as tabelas dos atos precisamos agora executar uma limpeza nas tebelas.

> ATENÇÃO: ESSE TRATAMENTO DE DADOS SERVE SOMENTE PARA NOMEAÇÃO. CADA ATO DEVE TER SEU PRÓPRIO TRATAMENTO.

> TRATAMENTO INDIVIDUAIS SERÃO ADICIONADOS NO FUTURO.

In [6]:
nomeacao_df = atos_csv_dict['Ato_Nomeacao_Comissionado']

nomeacao_fields = ['nome', 'cargo_efetivo', 'matricula', 'matricula_SIAPE', 'simbolo', 'cargo_comissionado', 'hierarquia_lotacao', 'orgao', 'Ato_Nomeacao_Comissionado']

# Nessa linha todas as colunas que não pertencem a nomecao são extraidas para uma segunda lista. (Compreensão de listas).
nomeacao_non_fields = [column for column in nomeacao_df.columns if column not in nomeacao_fields]

# Exclusão de todas as linhas que possuam algum valor nos campos que não pertecem a nomeacao.
nomeacao_df = nomeacao_df[nomeacao_df[nomeacao_non_fields].isna().any(axis=1)]

# Exclusão de todas as colunas que não pertecem a nomeação.
nomeacao_df = nomeacao_df.drop(columns=nomeacao_non_fields)

# Exclusão das linhas que não possuem anotação de atos.
nomeacao_df = nomeacao_df.dropna(subset=['Ato_Nomeacao_Comissionado'])

atos_csv_dict['Ato_Nomeacao_Comissionado'] = nomeacao_df
atos_csv_dict['Ato_Nomeacao_Comissionado']

,Ato_Nomeacao_Comissionado,cargo_comissionado,cargo_efetivo,hierarquia_lotacao,matricula,nome,orgao,simbolo,matricula_SIAPE
0,"NOMEAR o TC QOPM JOAQUIM SINESIO MARQUES, matr...",Assessor,TC QOPM,"Assessoria Tecnica, \nda Governadoria do Distr...",50.114-X,JOAQUIM SINESIO MARQUES,19 de janeiro de 2009,CNE-06,NaN
1,"NOMEAR o MAJ QOPM DOUGLAS CAMPOS MACHADO, matr...",Assessor,MAJ QOPM,Subsecretaria Administrativa,50.708-3,DOUGLAS CAMPOS MACHADO,Casa \nMilitar do Distrito Federal,DFA-12,NaN
2,NOMEAR BRUNA KAROLLYNE DIAS NASCIMENTO para ex...,Assessor,NaN,Subsecretaria de Administracao Geral,NaN,BRUNA KAROLLYNE DIAS NASCIMENTO,Secretaria de Estado \nde Publicidade Instituc...,DFA-14,NaN
3,NOMEAR ANA DONIZETE DE ASSIS para exercer o Ca...,Gerente,NaN,"Gerencia de Execucao de Obras, Conservacao e M...",NaN,ANA DONIZETE DE ASSIS,Governadoria do Distrito Federal,DFG-14,NaN
4,NOMEAR CHARLES DOUGLAS PROTAZIO SOUSA para exe...,Assessor,NaN,"Diretoria Social, da Administracao Regional de...",NaN,CHARLES DOUGLAS PROTAZIO SOUSA,Governadoria do Distrito Federal,DFA-11,NaN
...,...,...,...,...,...,...,...,...,...
7682,"NOMEAR ALESSANDRA NAZARE LEANDRO TAVARES, Tecn...",Chefe do Nucleo de Apoio \nOperacional,Tecnico Administrativo,"Gerencia do Centro de Saude no 04, da Diretori...",1.442.814-8,ALESSANDRA NAZARE LEANDRO TAVARES,\nSecretaria de Estado de Saude do Distrito Fe...,DFG-07,NaN
7683,"NOMEAR JOSE MOREIRA DANTAS, matricula 1.440.73...",Gerente,NaN,"Gerencia de Saude da Familia 1, da Diretoria R...",1.440.738-0,JOSE MOREIRA DANTAS,Secretaria de Estado de Saude do Distrito Federal,DFG-12,NaN
7684,"NOMEAR JANAINA MESSIAS GOMES, Auxiliar de Enfe...",Chefe do Nucleo de Apoio e Remo-\ncao de Paci...,Auxiliar de Enfermagem,"Gerencia de Enfermagem, da Diretoria do Hospit...",1.436.679-7,JANAINA MESSIAS GOMES,Secretaria de \nEstado de Saude do Distrito Fe...,DFG-07,NaN
7685,"NOMEAR MARCELO AUGUSTO SOARES DE LIMA, Carreir...",Chefe do Nucleo \nde Citopatologia e Anatomia ...,Carreira Medica (Anatomia Patologia),"Gerencia de Diagnose e Terapia, da Diretoria d...",190.079-X,MARCELO AUGUSTO SOARES DE LIMA,Secretaria de Estado de Saude do Distrito Federal,DFG-07,NaN


Um problema comum em todos os atos é a existência de valores *nan* no campo de texto do ato. Sem esse campo não é possível executar a etapa de criação de labels. Podemos assim realizar um laço que retirar todas as linhas que possuem esse campo com valor de *nan*.

In [7]:
for key in atos_csv_dict.keys():
    atos_csv_dict[key] = atos_csv_dict[key].dropna(subset=[key])

---
# 3. Criação dos labels IOB

### 3.1. Definiremos agora uma função *generate_labels* que recebe uma linha do csv do ato e adiciona o label de IOB.
> No formato IOB o caracter 'B' representa o início de uma entidade o 'I' uma parte de uma entidade q já tenha sido iniciada e 'O' um token que não possui label.
> Dessa forma adiciona-se o tipo da entidade em seguida do label identificado 'B-entidade'...'I-entidade'

In [8]:
# Altere aqui a forma de tokenizar as sentenças.
def tokenize(sentence):
    return nltk.word_tokenize(sentence)
#     tokenizer = nltk.RegexpTokenizer(r"\w+")
#     new_words = tokenizer.tokenize(sentence)
#     return new_words

def find_entity(row, token):
    for column in row.keys()[1:]:
        if row[column] is not numpy.nan and token == tokenize(row[column])[0]:
            return column
    
    return None

# Atualizar no futuro para qualquer ato.
# Aparentemente esse algoritmo está O(n*m) onde n é a quantidade de tokens e m a quantidade de colunas do df.
def generate_IOB_labels(row, key):
    labels = []
    entity_started = False
    for token in tokenize(row[key]):                         # Intera sobre cada token da anotação do ato.
        if not entity_started:                               # Caso uma entidade ainda n tenha sido identificada nos tokens.
            entity = find_entity(row, token)                 # Busca o token atual no primeiro token de todos os campos do df.
            if entity is not None:                           # Se foi encontrado o token no inicio de alguma entidade ele inicia a comparação token a token com a entidade.
                entity_started = True
                token_index = 1
                labels.append('B-' + entity)
            else:
                labels.append('O')
        else:                                                # Caso uma entidade já tenha sido identificada
            if token_index < len(tokenize(row[entity])) and token == tokenize(row[entity])[token_index]: # Checa se o próximo token pertence à entidade e se o tamanho da entidade chegou ao fim.
                labels.append('I-' + entity)                 # Se a entidade ainda possui tokens e a comparação foi bem sucedida adicione o label I.
                token_index += 1
                if token_index >= len(tokenize(row[entity])):
                    entity_started = False
            else:                                            # Se o token n for igual ou a entidade chegou ao fim.
                entity_started = False
                labels.append('O')
                
    return labels


### 3.2. Agora, com as funções de geração label prontas iremos criar uma lista de strings representando os labels para adicionar ao df de cada ato.

In [9]:
for key in atos_csv_dict.keys():
    labels_row = []
    for index, row in atos_csv_dict[key].iterrows():
        try:
            labels_row.append(' '.join(generate_IOB_labels(row, key)))
        except Exception as e:
            print(row)
            raise e
        
    atos_csv_dict[key].insert(len(atos_csv_dict[key].columns), 'labels_IOB', labels_row, False)

---
# 4. Criação das features e treinamento do CRF à moda José

> Faremos duas funções para gerar as features para o algoritmo CRF.

In [10]:
def extract_features(sentence):
  sentence_features = []
  for j in range(len(sentence)):
    word_feat = {
            'word': sentence[j].lower(),
            'capital_letter': sentence[j][0].isupper(),
            'all_capital': sentence[j].isupper(),
            'isdigit': sentence[j].isdigit(),
            'word_before': sentence[j].lower() if j==0 else sentence[j-1].lower(),
            'word_after:': sentence[j].lower() if j+1>=len(sentence) else sentence[j+1].lower(),
            'BOS': j==0,
            'EOS': j==len(sentence)-1
    }
    sentence_features.append(word_feat)
  return sentence_features

def separate_cols(arq, key):
    x = []
    y = []
    for index, row in arq.iterrows():
        x.append(extract_features(tokenize(row[key])))
        y.append(row['labels_IOB'].split())
    return x, y

> Aqui criaremos os dados usando 70% pra treinamento e 30% teste. Isso é feito para cada ato e ao final é apresentado a tebela do F1-Score

In [11]:
training_ratio = 0.7  

for key in atos_csv_dict.keys():
    print("------------------------------------------------------------------------------------")
    print("Ato:" + key)
    print("Tamanho: " + str(len(atos_csv_dict[key])))
    x, y = separate_cols(atos_csv_dict[key], key)
    train_x = x[:math.floor(training_ratio*len(atos_csv_dict[key]))]
    train_y = y[:math.floor(training_ratio*len(atos_csv_dict[key]))]
    test_x = x[math.floor(training_ratio*len(atos_csv_dict[key])):]
    test_y = y[math.floor(training_ratio*len(atos_csv_dict[key])):]

    len(atos_csv_dict[key]), len(train_x), len(train_y), len(test_x), len(test_y)

    model = sklearn_crfsuite.CRF(
        algorithm = 'l2sgd', 
        c2=1,
        max_iterations=100, 
        all_possible_transitions=True,
        verbose=False
    )

    model.fit(train_x, train_y)
    y_pred = model.predict(test_x)

    labels = list(model.classes_)
    labels.remove('O')

    f1 = metrics.flat_f1_score(test_y, y_pred, 
                          average='weighted', labels=labels)
    print(f1)
    print(metrics.flat_classification_report(
        test_y, y_pred, labels=labels, digits=3
    ))

------------------------------------------------------------------------------------
Ato:Ato_Retificacao_Comissionado
Tamanho: 50
0.3182178318515615
                        precision    recall  f1-score   support

      B-tipo_documento      0.923     0.800     0.857        15
    B-numero_documento      0.000     0.000     0.000         3
                B-nome      0.000     0.000     0.000         7
                I-nome      0.000     0.000     0.000        28
         B-numero_dodf      0.800     0.923     0.857        13
           B-data_dodf      0.000     0.000     0.000        15
           I-data_dodf      0.000     0.000     0.000        61
         B-pagina_dodf      0.000     0.000     0.000        15
B-informacao_corrigida      0.857     0.154     0.261        39
I-informacao_corrigida      0.904     0.314     0.466       239
      B-data_documento      0.393     0.733     0.512        15
      I-data_documento      0.389     0.733     0.509        60
            B-tipo